In [4]:
my_list = [1, 3, 5, 6, 7, 9, 10, 3]

In [5]:
def the_oracle(my_input):
    winner = 7
    return my_input is winner

In [6]:
for index, trial_number in enumerate(my_list):
    if the_oracle(trial_number) is True:
        print("Winner found at index: %i"%index)
        print("%i calls to the Oracle used"%(index+1))
        break

Winner found at index: 4
5 calls to the Oracle used


In [7]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# define the oracle circuit
oracle = QuantumCircuit(2, name="oracle")
oracle.cz(0,1)
oracle.to_gate()
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [9]:
backend = Aer.get_backend("statevector_simulator")
grover_circ = QuantumCircuit(2, 2)
grover_circ.h([0, 1])
grover_circ.append(oracle, [0, 1])
grover_circ.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c_0: ════════════════
                     
c_1: ════════════════

In [10]:
job = execute(grover_circ, backend)
result = job.result()

In [11]:
sv = result.get_statevector()
np.around(sv, 2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [12]:
reflection = QuantumCircuit(2, name="reflection")
reflection.h([0, 1])
reflection.z([0, 1])
reflection.cz(0, 1)
reflection.h([0, 1])
reflection.to_gate()

Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [13]:
reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [14]:
backend = Aer.get_backend("qasm_simulator")
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0, 1])
grover_circ.append(oracle, [0, 1])
grover_circ.append(reflection, [0, 1])
grover_circ.measure([0, 1], [0, 1])

In [15]:
grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c_0: ════════════════════════════════╩══╬═
                                        ║ 
c_1: ═══════════════════════════════════╩═

In [17]:
job = execute(grover_circ, backend, shots = 1)
result = job.result()
result.get_counts()

{'11': 1}